## R-CNN문제점
- RCNN
    - Input Image를 2000개 Extract 추출 
    - Warped 작업을 통해 CNN Image Square화 (이미지 변형으로인한 손실
    - Bounding Box Regression 작업 및 박스 위치 조정
    - SVM 작업을 통해 최종 이미지 분류 (해당 사물이 인지 아닌지)

- 문제점
    - Selective Search를 통해 뽑힌 2000개를 모두 CNN 연산해서 training, testing 시간이 오래걸림 
    - SVM 에서 수행한 Computation을 Share하지 않음 (BackPropagation이 안됨.)
    - 분류를 위한 SVM, 바운딩 박스를 위한 선형회귀까지 Multi-stage로 세 가지 모델을 모두 훈련시키기가 어려움

- RCNN : 2013년 11월
- SPP-Net : 2014년 6월
- Fast R-CNN : 2015년 4월

<hr>

## SPPNet 
- Spatial Pyramid Pooling (SPP) 
    - **FC Layer의 input이 고정인거지 CNN Input은 고정이 아니다**
    - **CNN에는 입력 이미지 크기, 비율 관계없이 Input으로 들어갈 수 있고 FC의 input size만 맞춰주면된다.**
    - Warping 작업(기준 이미지 크기에 맞추려고 하다보면 기존 위치정보를 잃게 됨)에서 일어나는 정보손실을 막으며, 이미지의 차원을 맞추기 위한 방법

![spp](img/spp.png)

- 방법
    - (1) 이미지를 CNN에 통과시켜 Feature map을 추출.
    - (2) 미리 정해진 4x4, 2x2, 1x1 영역의 피라미드로 Feature map 이미지를 분할함. (왼쪽부터 1x1, 2x2, 4x4)
    - (3) 각 한칸 마다 max pooling을 적용하여 하나의 값을 추출
    - (4) 최종적으로 해당 max값들을 추출(maxpooling)하여 3개의 피라미드 결과를 쭉 이어붙여 fixed vector 생성(Bag of Words)
    - (5) FC Layer의 Input으로 사용
- 요약
    - CNN을 통과한 Feautre map에서 2천개의 Region proposal을 만들고 region proposal마다 SPPNet에 집어넣어 고정된 크기의 Feature Vector 획득
- 일정 갯수의 지역으로 나눈뒤, 각 지역의 BoW(Bag of Words)를 적용
        
![Spatial_Pyramid_Pooling](img/Spatial_Pyramid_Pooling.png)

<hr>


## Fast RCNN

![FastRCNN2](img/FastRCNN2.png)


### 주요 아이디어 (ROI-pooling)
- SPP를 차용. Region of Interest Pooling을 사용
- 1개의 피라미드 (7x7) 만 사용
- **ROI-Pooling : 1개의 피라미드를 가진 SPP 방식을 통해, 고정된 크기의 Feature vector를 만드는 과정을 ROI Pooling 이라고 함**

- 방법
    - (1) 원본 이미지에서 Selective Search를 통한 2000개의 ROI 찾아냄
    - (2) 원본 이미지를 CNN을 통한 Feature Map 생성
    - (3) 해당 Feature Map들에 (1)에서 뽑은 ROI Projection (투영인듯)
    - (4) 각 특성맵들이 ROI에 의해 Projection 및 잘라짐 (부분으로 나눠짐)
    - (5) **해당 ROI를 7x7 ROI Pooling 적용**
        - 어떠한 Box등 가로세로를 일정한 Bin으로 나눔
        - 가장 큰 부분만 남게되고 최종적으로 이를 모두 붙여 고정된 fixed Vector 생성됨
    - (6) 고정된 fixed vector를 FC층에 입력 및 두개의 자녀 출력층을 가짐
        - Bbox Regressor를 통한 객체 위치 조정
        - Softmax를 통한 객체의 클래스 예측

![FastRCNN3](img/FastRCNN3.png)
 
<hr>

## 특징
- **end to end로 한번에 학습 가능**
    - 기존 (R-CNN)
        - Bounding box regression input은 cnn을 거치기전의 Region Proposal 데이터
        - SVM은 CNN을 거친 후의 Feature map이 들어가서 연산이 겹치지 않는다.

    - 현재 (Fast R-CNN)
        - ROI Pooling을 통한 RoI 영역(Region Proposal 같은)을 CNN이 거쳐진 Feature map에 투영 
        - **즉 동일 데이터가 softmax, bbox regressor로 들어가서 연산 공유**
![FastRCNN](img/FastRCNN.png)

<hr>

## 결론
- RoI Pooling을 통한 CNN 후의 region proposal 연산 반영
    - **(2000 region proposal) x (cnn연산) => (1 region proposal) x (cnn연산)**
    - **변경된 Feature vector가 기존의 region proposal을 projection 시킨 후 연산 한 것이므로 해당 output으로 classification과 bbox regression 학습 가능**
    
- 문제점
    - RoI를 생성하는 Selective Search 알고리즘은 **CNN 외부에서 진행**
        - -> 속도에 대한 Bottleneck
        

![RCNN_VS_FastRCNN](img/RCNN_VS_FastRCNN.PNG)

## 참고문헌 
- https://bskyvision.com/702 
- https://ganghee-lee.tistory.com/36
